In [1]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm
from meshparty import trimesh_io

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


Connecting celiib@10.28.0.34:3306


# AUTOMATICALLY GENERATING CODE FOR CONTACT PRE POST

In [3]:
compartment_type = {
"apical":2,
"basal":3,
"oblique":4,
"soma":5,
"axon":7,
"dendrites":8
}
#ones not doing:   "total":, "axon":


# total_length ="sum(postsyn_length * (majority_label=2))",
# n_syn ="sum( n_synapses * (majority_label=2))",
# n_contacts="sum(majority_label=2)",
# contact_conversion="sum( n_synapses * (majority_label=2))/sum(majority_label=2)",
# contact_density="sum( n_synapses * (majority_label=2))/sum( postsyn_length * (majority_label=2))"

# data_type = {
#     "total_length":["bigint unsigned","total postsynaptic contact length","sum( postsyn_length * (majority_label=","))"],
#     "n_syn":["bigint unsigned","number of synapses","sum( n_synapses * (majority_label=","))"],
#     "n_contacts":["bigint unsigned","number of contacts","sum(majority_label=",")"],
#     "contact_conversion":["float","synapse to contact ratio","sum( n_synapses * (majority_label=","))"
#                           ,"/sum(majority_label=",")"],
#     "contact_density":["float","synapse to contact length ratio","sum( n_synapses * (majority_label=","))/",
#                       "sum( postsyn_length * (majority_label=","))"]
# }

data_type = {
"n_contacts":["bigint unsigned","number of contacts","sum(majority_label=",")"],
"postsyn_length":["bigint unsigned","total postsynaptic contact length","sum( postsyn_length * (majority_label=","))"],
"contact_conversion":["float","synapse to contact ratio","if(sum(majority_label=",")=0,null,","sum( n_synapses * (majority_label=","))"
                  ,"/sum(majority_label=","))"],
"contact_density":["float","synapse to contact length ratio","if(sum( postsyn_length * (majority_label=","))=0,null,"
               ,"sum( n_synapses * (majority_label=","))/",
              "sum( postsyn_length * (majority_label=",")))"],
"n_synapses":["bigint unsigned","total number of synapses for neurite",
         "sum(n_synapses * (majority_label=","))"],
"synapse_sizes_mean":["float","average synaptic size","if((n_synapses * (majority_label=","))=0,0,","sum((n_synapses * (majority_label=","))","*synapse_sizes_mean*(majority_label=","))",
                                  "/sum(n_synapses * (majority_label=",")))"]
}



# synapses = sum(n_synapses)
# contacts = count(*)
# contact_conversion = sum(n_synapses)/count(*)
# contact_density = sum(n_synapses)/sum(post_syn length)
    
    #genereate the table header

# extra_headers = ["total"]
# for c in list(compartment_type.keys()) + extra_headers:
#     for keys,values in data_type.items():
#         print(str(c) + "_" + str(keys) + "   :" + str(values[0]) + " #" + str(c) + " " + str(values[1]))
        
# print("\n\n")

extra_headers = ["total"]
for c in extra_headers + list(compartment_type.keys()) :
    for keys,values in data_type.items():
        if values[0] != "float":
            print(str(c) + "_" + str(keys) + "   :" + str(values[0]) + " #" + str(c) + " " + str(values[1]))
        else:
            print(str(c) + "_" + str(keys) + "=null   :" + str(values[0]) + " #" + str(c) + " " + str(values[1]))
        
print("\n\n")


for keys_comp,values_comp in compartment_type.items():
    print("")
    for keys_data,values_data in data_type.items():
        base_string = str(keys_comp) + "_" + str(keys_data) + "= " 
        
        start_counter = 2
        rounds = (len(values_data)-2)/2
        base_string += '"'
        for i in range(0,int(rounds)):
            if keys_comp != "axon":
                base_string += values_data[start_counter] + str(values_comp) + values_data[start_counter + 1]
            else:
                base_string += values_data[start_counter] + "6 OR majority_label=7" + values_data[start_counter + 1]
            
            start_counter +=2
        base_string += '",'
        print(base_string)

total_n_contacts   :bigint unsigned #total number of contacts
total_postsyn_length   :bigint unsigned #total total postsynaptic contact length
total_contact_conversion=null   :float #total synapse to contact ratio
total_contact_density=null   :float #total synapse to contact length ratio
total_n_synapses   :bigint unsigned #total total number of synapses for neurite
total_synapse_sizes_mean=null   :float #total average synaptic size
apical_n_contacts   :bigint unsigned #apical number of contacts
apical_postsyn_length   :bigint unsigned #apical total postsynaptic contact length
apical_contact_conversion=null   :float #apical synapse to contact ratio
apical_contact_density=null   :float #apical synapse to contact length ratio
apical_n_synapses   :bigint unsigned #apical total number of synapses for neurite
apical_synapse_sizes_mean=null   :float #apical average synaptic size
basal_n_contacts   :bigint unsigned #basal number of contacts
basal_postsyn_length   :bigint unsigned #basal total

# MAKING PRE POST TEST TABLE

In [ ]:
#FinalPrePostTest.drop()

In [4]:
labels_to_keep = [2,3,4,5,6,7,8]

@schema
class FinalPrePostTest(dj.Computed):
    definition="""
    -> ta3p100.NeuriteRevised
    postsyn :bigint unsigned #id of the postsynaptic neuron
    ---
    total_n_contacts   :bigint unsigned #total number of contacts
    total_postsyn_length   :bigint unsigned #total total postsynaptic contact length
    total_contact_conversion=null   :float #total synapse to contact ratio
    total_contact_density=null   :float #total synapse to contact length ratio
    total_n_synapses   :bigint unsigned #total total number of synapses for neurite
    total_synapse_sizes_mean=null   :float #total average synaptic size
    apical_n_contacts   :bigint unsigned #apical number of contacts
    apical_postsyn_length   :bigint unsigned #apical total postsynaptic contact length
    apical_contact_conversion=null   :float #apical synapse to contact ratio
    apical_contact_density=null   :float #apical synapse to contact length ratio
    apical_n_synapses   :bigint unsigned #apical total number of synapses for neurite
    apical_synapse_sizes_mean=null   :float #apical average synaptic size
    basal_n_contacts   :bigint unsigned #basal number of contacts
    basal_postsyn_length   :bigint unsigned #basal total postsynaptic contact length
    basal_contact_conversion=null   :float #basal synapse to contact ratio
    basal_contact_density=null   :float #basal synapse to contact length ratio
    basal_n_synapses   :bigint unsigned #basal total number of synapses for neurite
    basal_synapse_sizes_mean=null   :float #basal average synaptic size
    oblique_n_contacts   :bigint unsigned #oblique number of contacts
    oblique_postsyn_length   :bigint unsigned #oblique total postsynaptic contact length
    oblique_contact_conversion=null   :float #oblique synapse to contact ratio
    oblique_contact_density=null   :float #oblique synapse to contact length ratio
    oblique_n_synapses   :bigint unsigned #oblique total number of synapses for neurite
    oblique_synapse_sizes_mean=null   :float #oblique average synaptic size
    soma_n_contacts   :bigint unsigned #soma number of contacts
    soma_postsyn_length   :bigint unsigned #soma total postsynaptic contact length
    soma_contact_conversion=null   :float #soma synapse to contact ratio
    soma_contact_density=null   :float #soma synapse to contact length ratio
    soma_n_synapses   :bigint unsigned #soma total number of synapses for neurite
    soma_synapse_sizes_mean=null   :float #soma average synaptic size
    axon_n_contacts   :bigint unsigned #axon number of contacts
    axon_postsyn_length   :bigint unsigned #axon total postsynaptic contact length
    axon_contact_conversion=null   :float #axon synapse to contact ratio
    axon_contact_density=null   :float #axon synapse to contact length ratio
    axon_n_synapses   :bigint unsigned #axon total number of synapses for neurite
    axon_synapse_sizes_mean=null   :float #axon average synaptic size
    dendrites_n_contacts   :bigint unsigned #dendrites number of contacts
    dendrites_postsyn_length   :bigint unsigned #dendrites total postsynaptic contact length
    dendrites_contact_conversion=null   :float #dendrites synapse to contact ratio
    dendrites_contact_density=null   :float #dendrites synapse to contact length ratio
    dendrites_n_synapses   :bigint unsigned #dendrites total number of synapses for neurite
    dendrites_synapse_sizes_mean=null   :float #dendrites average synaptic size
    """
    
    key_source = ta3p100.Segmentation & ta3p100.CurrentSegmentation
    
    
    def make(self,key):

        contact_pre_post = dj.U("segmentation","segment_id","postsyn").aggr(
            ta3p100.FinalContactTest.proj("postsyn","postsyn_length","n_synapses","majority_label","synapse_sizes_mean",segment_id="presyn")
            & [dict(majority_label=x) for x in labels_to_keep],

            
            total_n_contacts= "count(*)",
            total_postsyn_length= "sum(postsyn_length)",
            total_contact_conversion= "sum( n_synapses )/count(*)",
            total_contact_density= "sum( n_synapses )/sum(postsyn_length)",
            total_n_synapses= "sum(n_synapses)",
            total_synapse_sizes_mean= "sum(n_synapses*synapse_sizes_mean)/sum(n_synapses)",


            apical_n_contacts= "sum(majority_label=2)",
            apical_postsyn_length= "sum( postsyn_length * (majority_label=2))",
            apical_contact_conversion= "if(sum(majority_label=2)=0,null,sum( n_synapses * (majority_label=2))/sum(majority_label=2))",
            apical_contact_density= "if(sum( postsyn_length * (majority_label=2))=0,null,sum( n_synapses * (majority_label=2))/sum( postsyn_length * (majority_label=2)))",
            apical_n_synapses= "sum(n_synapses * (majority_label=2))",
            apical_synapse_sizes_mean= "if((n_synapses * (majority_label=2))=0,0,sum((n_synapses * (majority_label=2))*synapse_sizes_mean*(majority_label=2))/sum(n_synapses * (majority_label=2)))",

            basal_n_contacts= "sum(majority_label=3)",
            basal_postsyn_length= "sum( postsyn_length * (majority_label=3))",
            basal_contact_conversion= "if(sum(majority_label=3)=0,null,sum( n_synapses * (majority_label=3))/sum(majority_label=3))",
            basal_contact_density= "if(sum( postsyn_length * (majority_label=3))=0,null,sum( n_synapses * (majority_label=3))/sum( postsyn_length * (majority_label=3)))",
            basal_n_synapses= "sum(n_synapses * (majority_label=3))",
            basal_synapse_sizes_mean= "if((n_synapses * (majority_label=3))=0,0,sum((n_synapses * (majority_label=3))*synapse_sizes_mean*(majority_label=3))/sum(n_synapses * (majority_label=3)))",

            oblique_n_contacts= "sum(majority_label=4)",
            oblique_postsyn_length= "sum( postsyn_length * (majority_label=4))",
            oblique_contact_conversion= "if(sum(majority_label=4)=0,null,sum( n_synapses * (majority_label=4))/sum(majority_label=4))",
            oblique_contact_density= "if(sum( postsyn_length * (majority_label=4))=0,null,sum( n_synapses * (majority_label=4))/sum( postsyn_length * (majority_label=4)))",
            oblique_n_synapses= "sum(n_synapses * (majority_label=4))",
            oblique_synapse_sizes_mean= "if((n_synapses * (majority_label=4))=0,0,sum((n_synapses * (majority_label=4))*synapse_sizes_mean*(majority_label=4))/sum(n_synapses * (majority_label=4)))",

            soma_n_contacts= "sum(majority_label=5)",
            soma_postsyn_length= "sum( postsyn_length * (majority_label=5))",
            soma_contact_conversion= "if(sum(majority_label=5)=0,null,sum( n_synapses * (majority_label=5))/sum(majority_label=5))",
            soma_contact_density= "if(sum( postsyn_length * (majority_label=5))=0,null,sum( n_synapses * (majority_label=5))/sum( postsyn_length * (majority_label=5)))",
            soma_n_synapses= "sum(n_synapses * (majority_label=5))",
            soma_synapse_sizes_mean= "if((n_synapses * (majority_label=5))=0,0,sum((n_synapses * (majority_label=5))*synapse_sizes_mean*(majority_label=5))/sum(n_synapses * (majority_label=5)))",

            axon_n_contacts= "sum(majority_label=6 OR majority_label=7)",
            axon_postsyn_length= "sum( postsyn_length * (majority_label=6 OR majority_label=7))",
            axon_contact_conversion= "if(sum(majority_label=6 OR majority_label=7)=0,null,sum( n_synapses * (majority_label=6 OR majority_label=7))/sum(majority_label=6 OR majority_label=7))",
            axon_contact_density= "if(sum( postsyn_length * (majority_label=6 OR majority_label=7))=0,null,sum( n_synapses * (majority_label=6 OR majority_label=7))/sum( postsyn_length * (majority_label=6 OR majority_label=7)))",
            axon_n_synapses= "sum(n_synapses * (majority_label=6 OR majority_label=7))",
            axon_synapse_sizes_mean= "if((n_synapses * (majority_label=6 OR majority_label=7))=0,0,sum((n_synapses * (majority_label=6 OR majority_label=7))*synapse_sizes_mean*(majority_label=6 OR majority_label=7))/sum(n_synapses * (majority_label=6 OR majority_label=7)))",

            dendrites_n_contacts= "sum(majority_label=8)",
            dendrites_postsyn_length= "sum( postsyn_length * (majority_label=8))",
            dendrites_contact_conversion= "if(sum(majority_label=8)=0,null,sum( n_synapses * (majority_label=8))/sum(majority_label=8))",
            dendrites_contact_density= "if(sum( postsyn_length * (majority_label=8))=0,null,sum( n_synapses * (majority_label=8))/sum( postsyn_length * (majority_label=8)))",
            dendrites_n_synapses= "sum(n_synapses * (majority_label=8))",
            dendrites_synapse_sizes_mean= "if((n_synapses * (majority_label=8))=0,0,sum((n_synapses * (majority_label=8))*synapse_sizes_mean*(majority_label=8))/sum(n_synapses * (majority_label=8)))",
        )
        
        
        self.insert(contact_pre_post,skip_duplicates=True)

In [5]:
start_time = time.time()
FinalPrePostTest.populate()
print(f"Total time: {time.time()-start_time}")

Total time: 0.05910444259643555


In [6]:
FinalPrePostTest()

segmentation segmentation id,segment_id segment id unique within each Segmentation,postsyn id of the postsynaptic neuron,total_n_contacts total number of contacts,total_postsyn_length total total postsynaptic contact length,total_contact_conversion total synapse to contact ratio,total_contact_density total synapse to contact length ratio,total_n_synapses total total number of synapses for neurite,total_synapse_sizes_mean total average synaptic size,apical_n_contacts apical number of contacts,apical_postsyn_length apical total postsynaptic contact length,apical_contact_conversion apical synapse to contact ratio,apical_contact_density apical synapse to contact length ratio,apical_n_synapses apical total number of synapses for neurite,apical_synapse_sizes_mean apical average synaptic size,basal_n_contacts basal number of contacts,basal_postsyn_length basal total postsynaptic contact length,basal_contact_conversion basal synapse to contact ratio,basal_contact_density basal synapse to contact length ratio,basal_n_synapses basal total number of synapses for neurite,basal_synapse_sizes_mean basal average synaptic size,oblique_n_contacts oblique number of contacts,oblique_postsyn_length oblique total postsynaptic contact length,oblique_contact_conversion oblique synapse to contact ratio,oblique_contact_density oblique synapse to contact length ratio,oblique_n_synapses oblique total number of synapses for neurite,oblique_synapse_sizes_mean oblique average synaptic size,soma_n_contacts soma number of contacts,soma_postsyn_length soma total postsynaptic contact length,soma_contact_conversion soma synapse to contact ratio,soma_contact_density soma synapse to contact length ratio,soma_n_synapses soma total number of synapses for neurite,soma_synapse_sizes_mean soma average synaptic size,axon_n_contacts axon number of contacts,axon_postsyn_length axon total postsynaptic contact length,axon_contact_conversion axon synapse to contact ratio,axon_contact_density axon synapse to contact length ratio,axon_n_synapses axon total number of synapses for neurite,axon_synapse_sizes_mean axon average synaptic size,dendrites_n_contacts dendrites number of contacts,dendrites_postsyn_length dendrites total postsynaptic contact length,dendrites_contact_conversion dendrites synapse to contact ratio,dendrites_contact_density dendrites synapse to contact length ratio,dendrites_n_synapses dendrites total number of synapses for neurite,dendrites_synapse_sizes_mean dendrites average synaptic size
2,648518346341351467,648518346341351806,4,217,4.0,0.073658,16,28.7925,0,0,nan,nan,0,0.0,0,0,nan,nan,0,0.0,0,0,nan,nan,0,0.0,0,0,nan,nan,0,0.0,3,120,4.0,0.1,12,0.0,1,97,4.0,0.0411438,4,0.0
2,648518346341351467,648518346341351838,5,227,4.6,0.101348,23,75.4839,2,84,5.0,0.118751,10,0.0,0,0,nan,nan,0,0.0,1,71,1.0,0.0141443,1,0.0,0,0,nan,nan,0,0.0,2,72,6.0,0.166597,12,81.8167,0,0,nan,nan,0,0.0
2,648518346341351467,648518346341352006,13,674,4.46154,0.0860036,58,70.3538,4,206,5.5,0.106926,22,0.0,3,109,3.33333,0.0920556,10,0.0,1,27,5.0,0.186706,5,0.0,1,60,2.0,0.0330797,2,0.0,2,120,2.5,0.0416181,5,45.744,2,153,7.0,0.0917251,14,0.0
2,648518346341351467,648518346341352223,7,361,6.14286,0.119061,43,70.9263,2,143,5.5,0.0768103,11,0.0,1,63,10.0,0.159541,10,0.0,0,0,nan,nan,0,0.0,1,24,5.0,0.212404,5,0.0,3,132,5.66667,0.129052,17,65.7488,0,0,nan,nan,0,0.0
2,648518346341351467,648518346341353019,11,597,4.09091,0.0753403,45,66.7527,3,206,2.66667,0.0388123,8,0.0,4,125,5.25,0.168499,21,0.0,0,0,nan,nan,0,0.0,0,0,nan,nan,0,0.0,2,149,1.0,0.0133958,2,0.0,2,117,7.0,0.119413,14,71.2836
2,648518346341351467,648518346341353186,5,271,4.0,0.0738198,20,40.726,0,0,nan,nan,0,0.0,0,0,nan,nan,0,0.0,1,96,3.0,0.0313349,3,0.0,3,79,3.33333,0.125834,10,0.0,1,96,7.0,0.07313,7,13.47,0,0,nan,nan,0,0.0
2,648518346341351467,648518346341353574,5,373,4.6,0.0616969,23,74.7965,0,0,nan,nan,0,0.0,2,136,3.5,0.0512821,7,0.0,0,0,nan,nan,0,0.0,0,0,nan,nan,0,0.0,3,236,5.33333,0.0677134,16,71.6669,0,0,nan,nan,0,0.0
2,64851834634